<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#평가방식-MAE" data-toc-modified-id="평가방식-MAE-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>평가방식 MAE</a></span></li><li><span><a href="#library-import" data-toc-modified-id="library-import-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>library import</a></span></li><li><span><a href="#data-load" data-toc-modified-id="data-load-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>data load</a></span></li><li><span><a href="#전처리" data-toc-modified-id="전처리-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>전처리</a></span></li><li><span><a href="#feature-selection" data-toc-modified-id="feature-selection-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>feature selection</a></span></li><li><span><a href="#grid-search-cv" data-toc-modified-id="grid-search-cv-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>grid search cv</a></span></li></ul></div>

# 평가방식 MAE
... 회귀.... <br>
회귀도 input은 수치여야함.

# library import

In [1]:
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# data load

In [2]:
train = pd.read_csv('../235743_구내식당 식사 인원 예측 AI 경진대회_data/train.csv')
test = pd.read_csv('../235743_구내식당 식사 인원 예측 AI 경진대회_data/test.csv')

In [3]:
train.head(6)
# 일자
# 요일 ###
# 본사 정원수
# 본사휴가자수
# 본사출장자수
# 본사시간외근무명령서승인건수
# 현본사소속재택근무자수 ###
# 조식메뉴
# 중식메뉴 ###
# 석식메뉴 ###

# 중식계
# 석식계

# 중식계, 석식계는 합계인듯. 예측해야 하는 것.

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계
0,2016-02-01,월,2601,50,150,238,0.0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0
1,2016-02-02,화,2601,50,173,319,0.0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0
2,2016-02-03,수,2601,56,180,111,0.0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0
3,2016-02-04,목,2601,104,220,355,0.0,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",978.0,525.0
4,2016-02-05,금,2601,278,181,34,0.0,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",925.0,330.0
5,2016-02-11,목,2601,383,143,417,0.0,팬케익/찐빵 우유/두유/주스 계란후라이 견과류죽/쌀밥 (쌀:국내산) 감자찌개 ...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 시래기국 훈제오리구이 도토리묵무침 쌈무/양...","참치회덮밥 (쌀,현미흑미:국내산) 맑은국 군만두 과일샐러드 락교 포기김치 (...",1045.0,550.0


# 전처리

In [4]:
# train['조식메뉴'].value_counts()
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1205 entries, 0 to 1204
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   일자              1205 non-null   object 
 1   요일              1205 non-null   object 
 2   본사정원수           1205 non-null   int64  
 3   본사휴가자수          1205 non-null   int64  
 4   본사출장자수          1205 non-null   int64  
 5   본사시간외근무명령서승인건수  1205 non-null   int64  
 6   현본사소속재택근무자수     1205 non-null   float64
 7   조식메뉴            1205 non-null   object 
 8   중식메뉴            1205 non-null   object 
 9   석식메뉴            1205 non-null   object 
 10  중식계             1205 non-null   float64
 11  석식계             1205 non-null   float64
dtypes: float64(3), int64(4), object(5)
memory usage: 113.1+ KB


In [5]:
train.loc[(train['조식메뉴'].str.contains('쇠'))|(train['조식메뉴'].str.contains('소'))|(train['조식메뉴'].str.contains('고기'))|
          (train['조식메뉴'].str.contains('돼지'))|(train['조식메뉴'].str.contains('돈육'))|
          (train['조식메뉴'].str.contains('닭'))|(train['조식메뉴'].str.contains('치킨'))|
          (train['조식메뉴'].str.contains('오리')), '조식메뉴'] = 1
train.loc[train['조식메뉴']!=1, '조식메뉴'] = 0
test.loc[(test['조식메뉴'].str.contains('쇠'))|(test['조식메뉴'].str.contains('소'))|(test['조식메뉴'].str.contains('고기'))|
          (test['조식메뉴'].str.contains('돼지'))|(test['조식메뉴'].str.contains('돈육'))|
          (test['조식메뉴'].str.contains('닭'))|(test['조식메뉴'].str.contains('치킨'))|
          (test['조식메뉴'].str.contains('오리')), '조식메뉴'] = 1
test.loc[test['조식메뉴']!=1, '조식메뉴'] = 0

# int 형으로 바꾸기
train['조식메뉴'] = train['조식메뉴'].map(int)
test['조식메뉴'] = test['조식메뉴'].map(int)

In [6]:
# train[train['중식메뉴'].contains('고기')]
# (train.loc[train['중식메뉴'].str.contains('고기'), '중식메뉴']).count()
train.shape
# train.loc[train['중식메뉴'].str.contains('고기'), '중식메뉴']
train.loc[(train['중식메뉴'].str.contains('쇠'))|(train['중식메뉴'].str.contains('소'))|(train['중식메뉴'].str.contains('고기'))|
          (train['중식메뉴'].str.contains('돼지'))|(train['중식메뉴'].str.contains('돈육'))|
          (train['중식메뉴'].str.contains('닭'))|(train['중식메뉴'].str.contains('치킨'))|
          (train['중식메뉴'].str.contains('오리')), '중식메뉴'] = 1
train.loc[train['중식메뉴']!=1, '중식메뉴'] = 0
# train.loc[(train['중식메뉴'].str.contains('돼지'))|(train['중식메뉴'].str.contains('돈육')), '중식메뉴']
# train.loc[(train['중식메뉴'].str.contains('닭'))|(train['중식메뉴'].str.contains('치킨')), '중식메뉴']
# train.loc[(train['중식메뉴'].str.contains('오리')), '중식메뉴']

test.loc[(test['중식메뉴'].str.contains('쇠'))|(test['중식메뉴'].str.contains('소'))|(test['중식메뉴'].str.contains('고기'))|
          (test['중식메뉴'].str.contains('돼지'))|(test['중식메뉴'].str.contains('돈육'))|
          (test['중식메뉴'].str.contains('닭'))|(test['중식메뉴'].str.contains('치킨'))|
          (test['중식메뉴'].str.contains('오리')), '중식메뉴'] = 1
test.loc[test['중식메뉴']!=1, '중식메뉴'] = 0

# int 형으로 바꾸기
train['중식메뉴'] = train['중식메뉴'].map(int)
test['중식메뉴'] = test['중식메뉴'].map(int)

(1205, 12)

In [7]:
train.loc[(train['석식메뉴'].str.contains('쇠'))|(train['석식메뉴'].str.contains('소'))|(train['석식메뉴'].str.contains('고기'))|
          (train['석식메뉴'].str.contains('돼지'))|(train['석식메뉴'].str.contains('돈육'))|
          (train['석식메뉴'].str.contains('닭'))|(train['석식메뉴'].str.contains('치킨'))|
          (train['석식메뉴'].str.contains('오리')), '석식메뉴'] = 1
train.loc[train['석식메뉴']!=1, '석식메뉴'] = 0

test.loc[(test['석식메뉴'].str.contains('쇠'))|(test['석식메뉴'].str.contains('소'))|(test['석식메뉴'].str.contains('고기'))|
          (test['석식메뉴'].str.contains('돼지'))|(test['석식메뉴'].str.contains('돈육'))|
          (test['석식메뉴'].str.contains('닭'))|(test['석식메뉴'].str.contains('치킨'))|
          (test['석식메뉴'].str.contains('오리')), '석식메뉴'] = 1
test.loc[test['석식메뉴']!=1, '석식메뉴'] = 0

# int 형으로 바꾸기
train['석식메뉴'] = train['석식메뉴'].map(int)
test['석식메뉴'] = test['석식메뉴'].map(int)

In [8]:
train['요일'].values

train['요일'] = train['요일'].map({'월':1, '화':2, '수':3, '목':4, '금':5})
test['요일'] = test['요일'].map({'월':1, '화':2, '수':3, '목':4, '금':5})

train['요일']

array(['월', '화', '수', ..., '금', '월', '화'], dtype=object)

0       1
1       2
2       3
3       4
4       5
       ..
1200    3
1201    4
1202    5
1203    1
1204    2
Name: 요일, Length: 1205, dtype: int64

In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1205 entries, 0 to 1204
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   일자              1205 non-null   object 
 1   요일              1205 non-null   int64  
 2   본사정원수           1205 non-null   int64  
 3   본사휴가자수          1205 non-null   int64  
 4   본사출장자수          1205 non-null   int64  
 5   본사시간외근무명령서승인건수  1205 non-null   int64  
 6   현본사소속재택근무자수     1205 non-null   float64
 7   조식메뉴            1205 non-null   int64  
 8   중식메뉴            1205 non-null   int64  
 9   석식메뉴            1205 non-null   int64  
 10  중식계             1205 non-null   float64
 11  석식계             1205 non-null   float64
dtypes: float64(3), int64(8), object(1)
memory usage: 113.1+ KB


# feature selection

In [10]:
train.columns

Index(['일자', '요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '조식메뉴', '중식메뉴', '석식메뉴', '중식계', '석식계'],
      dtype='object')

In [14]:
X = train[['요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '조식메뉴', '중식메뉴', '석식메뉴']]
y_중식 = train[['중식계']]
y_석식 = train[['석식계']]

In [15]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
X_중식 = SelectKBest(chi2, k=2).fit_transform(X, y_중식)
X_중식.shape

X_석식 = SelectKBest(chi2, k=2).fit_transform(X, y_석식)
X_석식.shape

(1205, 2)

(1205, 2)

In [16]:
train.head()
train.tail()
X_중식 # ['본사시간외근무명령서승인건수', '현본사소속재택근무자수']
X_석식 # ['본사시간외근무명령서승인건수', '현본사소속재택근무자수']

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계
0,2016-02-01,1,2601,50,150,238,0.0,0,1,0,1039.0,331.0
1,2016-02-02,2,2601,50,173,319,0.0,0,1,1,867.0,560.0
2,2016-02-03,3,2601,56,180,111,0.0,0,1,1,1017.0,573.0
3,2016-02-04,4,2601,104,220,355,0.0,1,1,0,978.0,525.0
4,2016-02-05,5,2601,278,181,34,0.0,1,1,1,925.0,330.0


,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계
1200,2021-01-20,3,2983,75,198,4,391.0,0,1,1,1093.0,421.0
1201,2021-01-21,4,2983,92,231,462,351.0,1,0,1,832.0,353.0
1202,2021-01-22,5,2983,255,248,1,303.0,0,1,1,579.0,217.0
1203,2021-01-25,1,2983,107,153,616,327.0,1,1,1,1145.0,502.0
1204,2021-01-26,2,2983,69,183,551,362.0,0,1,1,1015.0,480.0


array([[238.,   0.],
       [319.,   0.],
       [111.,   0.],
       ...,
       [  1., 303.],
       [616., 327.],
       [551., 362.]])

array([[238.,   0.],
       [319.,   0.],
       [111.,   0.],
       ...,
       [  1., 303.],
       [616., 327.],
       [551., 362.]])

In [12]:
import lightgbm as lgbm
import random

# grid search cv

In [14]:
from sklearn.model_selection import GridSearchCV

parameters = [{'learning_rate': [0.1, 0.05, 0.01, 0.02], 'n_estimators': [100, 200, 500]}]

X_train_중식 = train[['요일', '현본사소속재택근무자수', '중식메뉴']]
X_train_석식 = train[['요일', '현본사소속재택근무자수', '석식메뉴']]

y_train_중식 = train[['중식계']]
y_train_석식 = train[['석식계']]

user_seed = 42
random.seed(user_seed)
LGBM = lgbm.LGBMRegressor(random_state = user_seed)
                          
grid_search = GridSearchCV(estimator = LGBM,
                           param_grid = parameters,
                           scoring = 'neg_mean_absolute_error',
                           cv = 10)

grid_search_중식 = grid_search.fit(X_train_중식, y_train_석식)
grid_search_석식 = grid_search.fit(X_train_석식, y_train_석식)

best_mae_중식 = grid_search_중식. best_score_
best_parameters_중식 = grid_search_중식. best_params_

best_mae_석식 = grid_search_석식. best_score_
best_parameters_석식 = grid_search_석식. best_params_

best_mae_중식, best_parameters_중식
best_mae_석식, best_parameters_석식

# [scoring method name](https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter)

(-81.634956846873, {'learning_rate': 0.05, 'n_estimators': 500})

(-81.634956846873, {'learning_rate': 0.05, 'n_estimators': 500})

In [15]:
X_train_중식 = train[['요일', '현본사소속재택근무자수', '중식메뉴']]
X_train_석식 = train[['요일', '현본사소속재택근무자수', '석식메뉴']]
y_train_중식 = train[['중식계']]
y_train_석식 = train[['석식계']]

X_test_중식 = test[['요일', '현본사소속재택근무자수', '중식메뉴']]
X_test_석식 = test[['요일', '현본사소속재택근무자수', '석식메뉴']]

user_seed = 42
random.seed(user_seed)
LGBM = lgbm.LGBMRegressor(random_state = user_seed, learning_rate = 0.05, n_estimators = 500)

model_중식 = LGBM.fit(X_train_중식, y_train_중식)
y_pred_test_중식 = model_중식.predict(X_test_중식, num_iteration = LGBM.best_iteration_)

model_석식 = LGBM.fit(X_train_석식, y_train_석식)
y_pred_test_석식 = model_석식.predict(X_test_석식, num_iteration = LGBM.best_iteration_)


In [16]:
중식계 = pd.DataFrame(y_pred_test_중식)
중식계.head()

,0
0,1059.528955
1,917.119868
2,760.368267
3,1170.908648
4,993.841448


In [17]:
석식계 = pd.DataFrame(y_pred_test_석식)
석식계.head()

,0
0,426.744744
1,462.530941
2,335.804193
3,558.728728
4,527.576167


In [18]:
findata = pd.concat([test['일자'], 중식계, 석식계], axis = 1)
findata.head()

,일자,0,0
0,2021-01-27,1059.528955,426.744744
1,2021-01-28,917.119868,462.530941
2,2021-01-29,760.368267,335.804193
3,2021-02-01,1170.908648,558.728728
4,2021-02-02,993.841448,527.576167


In [19]:
findata.columns = ['일자', '중식계', '석식계']
findata.head()

,일자,중식계,석식계
0,2021-01-27,1059.528955,426.744744
1,2021-01-28,917.119868,462.530941
2,2021-01-29,760.368267,335.804193
3,2021-02-01,1170.908648,558.728728
4,2021-02-02,993.841448,527.576167


In [20]:
findata.to_csv('6th grid search 요일, 재택자수, 중식메뉴, 석식메뉴.csv', index=False)